In [ ]:
import pandas as pd
import glob
from functools import reduce
from pathlib import Path
import pandas as pd
import os
from collections import defaultdict
import re
from datetime import timedelta
from tqdm import tqdm

In [ ]:
# Set your specific directory here
csv_dir = r'/Users/veronika/ownCloud/PCA_analysis2025/SocialOF/HFD/males/atg7KO/Single/rawdata' #load your raw deepof 0.8 csv data from a specific directory
file_list = glob.glob(f'{csv_dir}/*.csv')
output_path = r'/Users/veronika/ownCloud/PCA_analysis2025/SocialOF/HFD/males/atg7KO/Single' # Path to save the merged CSV file

print(f"Found {len(file_list)} CSV files in {csv_dir}") # this will print the number of CSV files found in the first specified directory - check if the files are loaded correctly


Found 20 CSV files in /Users/veronika/ownCloud/PCA_analysis2025/SocialOF/HFD/males/atg7KO/Single/rawdata


In [ ]:
##Merge all CSV files into a single DataFrame and renames the first column to "Time" and adds an "experimental_id" column with the file name (without extension)
dfs = []
for file in file_list:
    df = pd.read_csv(file)
    # Rename the first column to "Time"
    if len(df.columns) > 0:
        df.rename(columns={df.columns[0]: "Time"}, inplace=True)
    # Add experimental_id column with file name (without extension)
    df['experimental_id'] = file.split('/')[-1].split('.')[0]
    dfs.append(df)

master_df = pd.concat(dfs, ignore_index=True)

In [ ]:
out_file = os.path.join(output_path, 'master_combined_males_atg7KO_HFD_Single.csv') # Save the merged DataFrame to a CSV file
master_df.to_csv(out_file, index=False)
print(f"✅ Dataset saved to: {out_file}")

✅ Dataset saved to: /Users/veronika/ownCloud/PCA_analysis2025/SocialOF/HFD/males/atg7KO/Single/master_combined_males_atg7KO_HFD_Single.csv


In [ ]:
######################################### Set your directory and condition file paths
csv_file = Path(r'/Users/veronika/ownCloud/PCA_analysis2025/SocialOF/HFD/males/atg7KO/Single/master_combined_males_atg7KO_HFD_Single.csv') #this is the path to the merged CSV file, we created above
condition_file = Path(r'/Users/veronika/ownCloud/PCA_analysis2025/SocialOF/HFD/males/atg7KO/exp_condi_atg7KOmales_HFD.csv') #this is the path to the condition file, which contains the experimental conditions for each sample

In [29]:
# Load condition mapping file
condition_df = pd.read_csv(condition_file)

# Create genotype map based on experimental ID
geno_map = dict(zip(condition_df['experimental_id'], condition_df['Geno']))

In [30]:
# Load data file
df = pd.read_csv(csv_file)

In [ ]:
#Rename first column to "Time" if necessary - this is to ensure consistency
if df.columns[0] != 'Time':
    df.rename(columns={df.columns[0]: 'Time'}, inplace=True)

# Extract just the ID (e.g., "ID196") from the 'experimental_id' path
df['experimental_id'] = df['experimental_id'].apply(lambda x: Path(x).name.split('+')[0])

# Map to Geno
df['Geno'] = df['experimental_id'].map(geno_map).fillna('Unknown')

In [32]:
# Save final output
output_file = csv_file.parent / (csv_file.stem + '_FINAL.csv')
df.to_csv(output_file, index=False)

print(f"✅ Saved cleaned file to: {output_file}")

✅ Saved cleaned file to: /Users/veronika/ownCloud/PCA_analysis2025/SocialOF/HFD/males/atg7KO/Single/master_combined_males_atg7KO_HFD_Single_FINAL.csv


##  A little code check if experimental conditions were loaded correctly

In [ ]:
#This will print the unique genotypes found in the dataset - useful for debugging
unique_genotypes = df['Geno'].unique()
print(f"Unique genotypes found: {unique_genotypes}")

Unique genotypes found: ['control' 'atg7KO']


In [ ]:
# Another check to see if there are any unknown genotypes (should have been filled with 'Unknown' and already checked above)
geno_check = df[df["Geno"] == "Unknown"]
print(f"Number of unknown genotypes: {len(geno_check)}")

Number of unknown genotypes: 0


#   Proceed only if you have anytroubles with geno matching or time in your dataset

Checking for the Unknown genotype and to what experimental ID it matches - if there are inconsisencies in loading, check if the experimental file at the beginning is formatted correctly

In [35]:
# Filter the dataframe for rows where genotype is 'Unknown'
unknown_geno_df = df[df['Geno'] == 'Unknown']

# List all unique experimental_id values with Unknown genotype
experimental_ids_with_unknown = unknown_geno_df['experimental_id'].unique()

print(experimental_ids_with_unknown)

[]


Checking if the Time corresponds to a lenght we would expect - e.g. no funny timestamps - it should look like e.g.: '00:00:00' ... '00:09:59.0665'

In [34]:
unique_Times = df.sort_values(['Time', 'experimental_id']).reset_index(drop=True)
unique_Times = unique_Times['Time'].unique()
print(f"Unique time points found: {unique_Times}")

Unique time points found: ['00:00:00' '00:00:00.066563296' '00:00:00.066566754' ...
 '00:10:01.866828890' '00:10:01.933348095' '00:10:01.933414445']


Checking the whole dataset as a table - if the behaviours look right, experimental id make sense and the mapping to Geno looks ok - check the Time column here!! Excel can be funny and additional forcing to the right format might be neccessary

In [35]:
df_sorted = df.sort_values(by=['experimental_id', 'Time']).reset_index(drop=True)
df_sorted.tail()  # Display the first few rows of the sorted DataFrame

,Time,climb-arena,sniff-arena,immobility,stat-lookaround,stat-active,stat-passive,moving,sniffing,speed,missing,experimental_id,Geno
187582,00:10:00.667036011,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,53.595,0,ID95,atg7KO
187583,00:10:00.733628808,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,47.490,0,ID95,atg7KO
187584,00:10:00.800221606,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,39.690,0,ID95,atg7KO
187585,00:10:00.866814404,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,27.660,0,ID95,atg7KO
187586,00:10:00.933407202,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,13.830,0,ID95,atg7KO
